In [8]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
from datetime import datetime, timedelta

# 1. Definisikan ulang Class Model (Harus SAMA PERSIS dengan file 03)
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# 2. Inisialisasi Model Kosong
model = LSTMModel()

# 3. Load "Otak" yang sudah pintar (Weights)
model.load_state_dict(torch.load('lstm_stock_model.pth'))
model.eval() # Set ke mode evaluasi (bukan latihan lagi)
print("Model berhasil di-load! Siap meramal.")

Model berhasil di-load! Siap meramal.


In [9]:
# Ambil data 1 tahun terakhir sampai detik ini
symbol = "BBRI.JK"
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

print(f"Mengambil data real-time {symbol} dari {start_date} sampai {end_date}...")
df_new = yf.download(symbol, start=start_date, end=end_date)

# Bersihkan header multi-index (masalah yfinance yg tadi)
if isinstance(df_new.columns, pd.MultiIndex):
    df_new.columns = df_new.columns.get_level_values(0)

df_new = df_new.filter(['Close']).dropna()

print(f"Harga penutupan terakhir ({df_new.index[-1].date()}): Rp {df_new['Close'].values[-1]}")
df_new.tail()

[*********************100%***********************]  1 of 1 completed

Mengambil data real-time BBRI.JK dari 2025-01-19 sampai 2026-01-19...
Harga penutupan terakhir (2026-01-15): Rp 3820.0


,Close
Date,
2026-01-09,3680.0
2026-01-12,3700.0
2026-01-13,3720.0
2026-01-14,3720.0
2026-01-15,3820.0


In [10]:
# 1. Scaling (Samakan logikanya dengan training)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_new.values)

# 2. Ambil 60 hari terakhir
# Logic: Kita butuh sequence terakhir [H-60, ... H-0]
last_60_days = scaled_data[-60:] 

# 3. Ubah jadi Tensor [1, 60, 1]
# Cukup unsqueeze(0) saja karena data aslinya sudah punya dimensi fitur (kolom)
X_input = torch.from_numpy(last_60_days).type(torch.Tensor).unsqueeze(0)

print(f"Shape Input: {X_input.shape}") # HARUSNYA KELUAR: [1, 60, 1]

Shape Input: torch.Size([1, 60, 1])


In [11]:
# Matikan gradien (hemat memori)
with torch.no_grad():
    # Prediksi (Outputnya masih angka 0-1)
    predicted_scaled = model(X_input).numpy()

# Kembalikan ke harga Rupiah asli
predicted_price = scaler.inverse_transform(predicted_scaled)

last_real_price = df_new['Close'].values[-1]
future_price = predicted_price[0][0]

print("="*40)
print(f"Harga Terakhir (Real): IDR {last_real_price:.2f}")
print(f"Prediksi BESOK (AI)  : IDR {future_price:.2f}")
print("="*40)

# Cek naik atau turun?
diff = future_price - last_real_price
persen = (diff / last_real_price) * 100

if diff > 0:
    print(f"Kesimpulan: BULLISH 🚀 (Potensi naik {persen:.2f}%)")
else:
    print(f"Kesimpulan: BEARISH 🔻 (Potensi turun {persen:.2f}%)")

Harga Terakhir (Real): IDR 3820.00
Prediksi BESOK (AI)  : IDR 3740.92
Kesimpulan: BEARISH 🔻 (Potensi turun -2.07%)
